<a href="https://colab.research.google.com/github/Enjeru-105/Google-Colab/blob/main/Bloc_de_notas_con_Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 104.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.1 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st

class Nota:
  def __init__(self, titulo, contenido):
    self.titulo = titulo
    self.contenido = contenido
    self.tipo = 'base'

  def mostrar(self):
    return f"**Titulo:** {self.titulo}\n\n**Contenido:** {self.contenido}\n\n"

  def modificar_t(self, titulo):
    self.titulo = titulo

  def modificar_c(self, contenido):
    self.contenido = contenido

class NotaTexto(Nota):
  def __init__(self, titulo, contenido):
    super().__init__(titulo, contenido)
    self.tipo = 'texto'

  def mostrar(self):
    return super().mostrar()


class NotaLista(Nota):
  def __init__(self, titulo, contenido=None):
    contenido = contenido if contenido is not None else []
    super().__init__(titulo, contenido)
    self.tipo = 'lista'

  def mostrar(self):
    texto = f"**Titulo:** {self.titulo}\n\n**Contenido (Lista):** \n\n"
    for i, item in enumerate(self.contenido, 1):
      texto += f"{i}. {item}\n"
    return texto + "\n"


class NotaImagen(Nota):
  def __init__(self, titulo, contenido):
    super().__init__(titulo, contenido)
    self.tipo = 'imagen'
    #Contenido es una URL

  def mostrar(self):
    return f"**Titulo:** {self.titulo}\n\n![Imagen]({self.contenido})\n\n"

class BlocDeNotas:
  def __init__(self):
    self.notas = []

  def agregar_nota(self, nota):
    self.notas.append(nota)

  def existe_nota(self, titulo):
    return any(nota.titulo == titulo for nota in self.notas)

  def obtener_nota(self, titulo):
    for nota in self.notas:
      if nota.titulo == titulo:
        return nota
    return None

  def eliminar_todas(self):
    self.notas = []

  #Esta función la tuve que poner porque investigué y hay
  #Como un error en colab que algunas listas las puede serializar
  #o algo así como si fueran diccionarios y daba errores al editarlas
  def refrescar(self):
    for i, nota in enumerate(self.notas):
      if isinstance(nota, dict):
        tipo = nota.get("Tipo", "Texto")
        titulo = nota.get("Titulo", "")
        contenido = nota.get("Contenido", "")
        if tipo == "lista":
          self.notas[i] = NotaLista(titulo, contenido)
        elif tipo == "imagen":
          self.notas[i] = NotaImagen(titulo, contenido)
        else:
          self.notas[i] = NotaTexto(titulo, contenido)



#-------------------------main-----------------------------#
if 'bloc' not in st.session_state:
  st.session_state.bloc = BlocDeNotas()
else:
  st.session_state.bloc.refrescar()

if 'opcion' not in st.session_state:
  st.session_state.opcion = 'Mostrar Todas las Notas'

st.title('🗒️Bloc de Notas')
opciones = ["📝Agregar Nota", "🔎Mostrar todas las Notas", "✍️Editar Nota", "🗑️Eliminar Todas las Notas"]
st.session_state.opcion = st.sidebar.radio("¿Que desea hacer?", opciones)

#agragar nota
if st.session_state.opcion == "📝Agregar Nota":
  st.subheader("📝Agregar Nota")
  tipo = st.selectbox("Seleccione el tipo de nota:", ["Nota de Texto", "Nota de Lista", "Nota de Imagen"])

  with st.form("form_agregar"):
    titulo = st.text_input("Titulo de la nota:")
    if tipo == "Nota de Texto":
      contenido = st.text_area("Contenido de la nota:")
    elif tipo == "Nota de Lista":
      lista_texto = st.text_area("Elementos de la lista (Uno por línea):")
      contenido = [item.strip() for item in lista_texto.splitlines() if item.strip()]
    else:
      contenido = st.text_input("URL de la imagen:")

    submit_agregar = st.form_submit_button("Guardar Nota")
    if submit_agregar:
      if not titulo:
        st.error("Debe ingresar un título a la nota.")
      elif st.session_state.bloc.existe_nota(titulo):
        st.error("Ya existe una nota con ese título, ingrese otro")
      else:
        if tipo == "Nota de Texto":
          nota = NotaTexto(titulo, contenido)
        elif tipo == "Nota de Lista":
          nota = NotaLista(titulo, contenido)
        else:
          nota = NotaImagen(titulo, contenido)
        st.session_state.bloc.agregar_nota(nota)
        st.success("Nota agregada con éxito")

#Mostrar todas las notas
elif st.session_state.opcion == "🔎Mostrar todas las Notas":
  st.subheader("🗒️Notas Guardadas")
  if not st.session_state.bloc.notas:
    st.info("❌No hay notas guardadas aún")
  else:
    for nota in st.session_state.bloc.notas:
      st.markdown(nota.mostrar())
      st.markdown("---")

#Editar Notas
elif st.session_state.opcion == "✍️Editar Nota":
  st.subheader("✍️Editar Nota")
  if not st.session_state.bloc.notas:
    st.info("❌No hay notas guardadas aún")
  else:
    titulo = st.selectbox("Seleccione la nota a editar", [nota.titulo for nota in st.session_state.bloc.notas])
    nota = st.session_state.bloc.obtener_nota(titulo)

    with st.form(key = "form_editar"):
      nuevo_titulo = st.text_input("Nuevo título:", nota.titulo)
      if nota.tipo == "texto":
        nuevo_contenido = st.text_area("Nuevo contenido:", nota.contenido)
      elif nota.tipo == "lista":
        lista_actual = "\n".join(nota.contenido) if isinstance(nota.contenido, list) else ""
        nuevo_texto_lista = st.text_area("Ingrese los nuevos elementos de la lista (Uno por línea):", value = lista_actual)
        nuevo_contenido = [item.strip() for item in nuevo_texto_lista.splitlines() if item.strip()]
      elif nota.tipo == "imagen":
        nuevo_contenido = st.text_input("Ingrese la nueva URL de la imagen:", nota.contenido)
      else:
        st.error("Tipo de nota no soportado")
        st.stop()

      submit_editar = st.form_submit_button("Guardar Cambios")

      if submit_editar:
        if nuevo_titulo != nota.titulo:
          if st.session_state.bloc.existe_nota(nuevo_titulo):
            st.error("Ya existe una nota con ese título, ingrese otro")
          else:
            nota.modificar_t(nuevo_titulo)

        nota.modificar_c(nuevo_contenido)
        st.success("Cambios guardados con éxito")
        st.rerun()

elif st.session_state.opcion == "🗑️Eliminar Todas las Notas":
  st.subheader("🗑️Eliminar Todas las Notas")
  if not st.session_state.bloc.notas:
    st.info("❌No hay notas guardadas aún")
  elif st.button("⚠️Eliminar Todas las Notas⚠️"):
    st.session_state.bloc.eliminar_todas()
    st.success("🗑️Todas las notas han sido eliminadas")
    st.rerun()

Overwriting app.py


In [ ]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸
added 22 packages in 7s
⠸
⠸3 packages are looking for funding
⠸  run `npm fund` for details
⠸

In [ ]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.106.3.237
⠙⠹your url is: https://warm-pianos-wave.loca.lt
